In [ ]:
from pymongo import MongoClient
import matplotlib.pyplot as plt
import numpy as np

client = MongoClient()
collection_pc = client['bigdata']['peoplecount']
collection_co = client['bigdata']['corona-deutschland-neue-faelle']

In [ ]:
query_pc = collection_pc.aggregate([
    {
        '$group': {
            '_id': {
                '$dateToString': {
                    'date': '$day',
                    'format': '%Y-%U'
                }
            },
            'count': {
                '$sum': '$count'
            }
        }
    },
    {
        '$sort': {
            '_id': 1
        }
    }
])
aggregate_pc = list(query_pc)

In [ ]:
query_co = collection_co.aggregate([
    {
        '$group': {
            '_id': {
                '$dateToString': {
                    'date': '$Date', 
                    'format': '%Y-%U'
                }
            }, 
            'neueFaelle': {
                '$sum': '$neueFaelle'
            }
        }
    }, {
        '$sort': {
            '_id': 1
        }
    }
])
aggregate_co = list(query_co)

In [ ]:
lasts = [x for x in aggregate_pc if x['_id'].endswith('52')]
firsts = [x for x in aggregate_pc if x['_id'].endswith('00')]

In [ ]:
for i in range(2):
    lasts[i]['count'] += firsts[i]['count']
aggregate_pc = [x for x in aggregate_pc if not x['_id'].endswith('00')]

In [ ]:
first_part_pc = [x for x in aggregate_pc if '2018-44' <= x['_id'] <= '2019-43']
second_part_pc = [x for x in aggregate_pc if '2019-44' <= x['_id'] <= '2020-43']

In [ ]:
x_pc = [x['_id'][5:] for x in first_part_pc]
y_pc_1s = [y['count'] for y in first_part_pc]
y_pc_2s = [y['count'] for y in second_part_pc]

In [ ]:
x_co = [x['_id'][5:] for x in aggregate_co]
y_co = [next((y['neueFaelle'] for y in aggregate_co if y['_id'][5:] == x and x <= '43'), 0) for x in x_pc]

In [ ]:
# Funktion um zwei Skalen anzugleichen (Antwort vom Stackoverflow: https://stackoverflow.com/questions/10481990/matplotlib-axis-with-two-scales-shared-origin)
def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    inv = ax2.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, y1-y2))
    miny, maxy = ax2.get_ylim()
    ax2.set_ylim(miny+dy, maxy+dy)

In [ ]:
f, axs = plt.subplots(nrows = 2, figsize = (14,8))

# Zeichnen des oberen Diagramms
ax1 = axs[0]
ax1.ticklabel_format(style = 'plain')
ax1.plot(x_pc, y_pc_1s, label = first_part_pc[0]['_id'] + ' - ' + first_part_pc[-1]['_id'])
ax1.plot(x_pc, y_pc_2s, label = second_part_pc[0]['_id'] + ' - ' + second_part_pc[-1]['_id'])

# Zeichnen des unteren Diagramms
ax2 = axs[1]
# Differenz der Passantenfrequenzen
ax2.plot(x_pc, np.array(y1s) - np.array(y2s), label = 'Differenz', color = 'green')
# Kopieren der X-Achse, um zweite Y-Achse zu erhalten (unterschiedliche Skalierung)
ax2_co = ax2.twinx()
ax2_co.plot(y_co, label = 'Corona-Fallzahlen', color = 'red')

# Zeichnen der drei Legenden
ax1.legend()
ax2.legend(loc = 'upper left')
ax2_co.legend(loc = 'upper right')

# Entfernen der oberen Linien der beiden Diagramme
ax1.spines['top'].set_color('none')
ax2.spines['top'].set_color('none')
ax2_co.spines['top'].set_color('none')

# Zeichnen einer Linie durch y = 0 im zweiten Diagramm
ax2.axhline(0, color='black')

# Beschriften der Achsen Diagramm 1
ax1.set_ylabel('Passantenfrequenz')
ax1.set_xlabel('Kalenderwoche')

# Beschriften der Achsen Diagramm 2
ax2.set_ylabel('Differenz der Passantenfrequenz')
ax2_co.set_ylabel('Corona-Fallzahlen')
ax2.set_xlabel('Kalenderwoche')

# Ansonsten ist die rechte Achse abgeschnitten
f.tight_layout()
# Angleichen der 0 im zweiten Diagramm
align_yaxis(ax2, 0, ax2_co, 0)

# Zeigen der Diagramme
plt.show()